### mic被占用测试

In [6]:
import re
audio_record = []

def check_micphone(file_path):
    with open(file_path, encoding='utf-8', errors='ignore') as file:
        log_lines = file.readlines()

    audio_packageName_record = {}
    issues = []
    packageName_pattern = re.compile(r'packageName: (\S+)')
    
    # 遍历日志
    for line in log_lines:
        if re.search('AudioRecord: start', line) is not None:
            audio_record.append(line)
            start_package = packageName_pattern.search(line)
            if start_package:
                # audio_packageName_record.append(line)
                package_name = start_package.group(1)
                if package_name not in audio_packageName_record:
                    audio_packageName_record[package_name] = []
                audio_packageName_record[package_name].append(line)
                
            
        elif re.search('AudioRecord: stop', line) is not None:
            audio_record.append(line)
            stop_package = packageName_pattern.search(line)
            if stop_package:
                # audio_packageName_record.append(line)
                package_name = stop_package.group(1)
                if package_name in audio_packageName_record and audio_packageName_record[package_name]:
                    audio_packageName_record[package_name].pop(0)
                # else:
                #     issues.append(line)
        
    # 收集所有没有对应stop的start日志
    for package_name, starts in audio_packageName_record.items():
        for start in starts:
            issues.append(start)

    return issues

def main():
    # 日志文件路径
    file_path = 'microphone_test.txt'  # 修改为你的日志文件路径
    # 检查麦克风问题
    mic_issues = check_micphone(file_path)
    # 输出结果
    if mic_issues:
        print("麦克风问题：")
        for issue in mic_issues:
            print(issue)
    else:
        print("未发现麦克风问题")

    print("*******")
    for item in audio_record:
            print(item)

if __name__ == '__main__':
    main()


麦克风问题：
07-15 14:24:43.872  1000  4387  4387 D AudioRecord: start(98): 0xb400007443bcba60, sync event 0 trigger session 0 packageName: com.mediatek.voicecommand

07-15 14:24:43.872  1000  4387  4387 D AudioRecord: start(98): 0xb400007443bcba60, sync event 0 trigger session 0 packageName: com.mediatek.voicecommand

*******
07-15 14:24:28.970 10184  9250 28745 D AudioRecord: start(94): return status 0

07-15 14:24:29.682 10184  9250 28745 D AudioRecord: stop(94): 0xb400007443c8f300, mActive: 1  packageName: com.miui.voiceassist

07-15 14:24:29.780 10184  9250 28745 D AudioRecord: stop(94) done

07-15 14:24:29.797 10184  9250 28745 D AudioRecord: stop(94): 0xb400007443c8f300, mActive: 0  packageName: com.miui.voiceassist

07-15 14:24:29.798 10184  9250 28745 D AudioRecord: stop(94): 0xb400007443c8f300, mActive: 0  packageName: com.miui.voiceassist

07-15 14:24:43.872  1000  4387  4387 D AudioRecord: start(98): 0xb400007443bcba60, sync event 0 trigger session 0 packageName: com.mediatek.voi

### speaker增益导致失败

In [7]:
import re

def check_audiotrack(file_path):
    with open(file_path, encoding='utf-8', errors='ignore') as file:
        log_lines = file.readlines()

    audio_packageName_track = {}
    audio_track = []
    issues_track = []
    packageName_pattern = re.compile(r'packageName: (\S+)')

    for line in log_lines:
        if re.search('AudioTrack: start', line) is not None:
            audio_track.append(line)
            start_package = packageName_pattern.search(line)
            if start_package:
                package_name = start_package.group(1)
                if package_name not in audio_packageName_track:
                    audio_packageName_track[package_name] = []
                audio_packageName_track[package_name].append(line)
        elif re.search('AudioTrack: stop', line) is not None:
            audio_track.append(line)
            stop_package = packageName_pattern.search(line)
            if stop_package:
                package_name = stop_package.group(1)
                if package_name in audio_packageName_track and audio_packageName_track[package_name]:
                    audio_packageName_track[package_name].pop(0)
                # else:
                #     issues_track.append(line)

    # 收集所有没有对应stop的start日志
    for package_name, starts in audio_packageName_track.items():
        for start in starts:
            issues_track.append(start)

    return issues_track

def main():
    # 日志文件路径
    file_path = 'audiotrack_test.txt'  # 修改为你的日志文件路径

    # 检查 AudioTrack 问题
    audiotrack_issues = check_audiotrack(file_path)

    # 输出结果
    if audiotrack_issues:
        print("AudioTrack 问题：")
        for issue in audiotrack_issues:
            print(issue)
    else:
        print("未发现 AudioTrack 问题")

if __name__ == '__main__':
    main()


未发现 AudioTrack 问题


### 读取文件夹下符合条件的txt文件

In [8]:
import os
import glob

# 输入的文件路径，假设已知
file_path = "D:\\2024-07-07-153720-67921922\\AAAVoicetriggerDump\\bugreport-goku-UKQ1.240116.001-2024-07-07-15-32-12.txt"

# 提取上一级目录
directory = os.path.dirname(file_path)

# 查找目录下所有的 .txt 文件，排除以 bugreport- 开头的文件
txt_files = glob.glob(os.path.join(directory, '*.txt'))
txt_files = [f for f in txt_files if not os.path.basename(f).startswith('bugreport-')]

# 合并所有符合条件的 .txt 文件内容
combined_log_lines = []
for file in txt_files:
    with open(file, 'r', encoding='utf-8', errors='ignore') as log_file:
        combined_log_lines.extend(log_file.readlines())

# 将合并的内容保存到一个新的文件中
combined_file_path = os.path.join(directory, 'combined_logs.txt')
with open(combined_file_path, 'w', encoding='utf-8', errors='ignore') as combined_file:
    combined_file.writelines(combined_log_lines)

print(f"合并的日志已保存到 {combined_file_path}")


合并的日志已保存到 D:\2024-07-07-153720-67921922\AAAVoicetriggerDump\combined_logs.txt
